In [3]:
import csv
from collections import OrderedDict
import tqdm

In [4]:

#"""
with open("train.csv") as training_csv:
    train_list_tmp = csv.reader(training_csv)
    train_list = []
    for row in train_list_tmp:
        train_list.append(row)
    train_list = train_list[1:]#"""
with open("test.csv") as test_csv:
    test_list_tmp = csv.reader(test_csv)
    test_list = []
    for row in test_list_tmp:
        test_list.append(row)
    test_list = test_list[1:]

In [49]:
all_features = []
for row in tqdm.tqdm(train_list):
    features = OrderedDict()
    features['battleneturl'] = row[0].split(';')[0]
    features['race'] =  row[0].split(';')[1]
    features['s'] = 0
    features['sBase'] = 0
    features['sMineral'] = 0
    for i in range(0,10):
        for j in range (0,10):
            features['hotkey%s%s'%(i,j)]= 0
    features['apm'] = 0
    lastFrame = 1
    for val in row[1:]:
        if val == "sBase":
            features['sBase'] = features.get("sBase",0) + 1
        elif val == "s":
            features['s'] = features.get("s",0) + 1
        elif val == "sMineral":
            features['sMineral'] = features.get("sMineral",0) + 1
        elif 'hotkey' in val:
            features[val] = features.get(val,0) + 1
        else:
            features['apm'] = features.get("apm",0) + 1
            lastFrame = int(val)
    features['apm'] = features['apm'] / (lastFrame/60)
    #print(features)
    all_features.append(features)

100%|██████████| 4099/4099 [02:07<00:00, 32.07it/s]


In [5]:
test_features = []
for row in tqdm.tqdm(test_list):
    features = OrderedDict()
    features['row ID'] = row[0].split(';')[0]
    features['race'] =  row[0].split(';')[1]
    features['s'] = 0
    features['sBase'] = 0
    features['sMineral'] = 0
    for i in range(0,10):
        for j in range (0,10):
            features['hotkey%s%s'%(i,j)]= 0
    features['apm'] = 0
    lastFrame = 1
    for val in row[1:]:
        if val == "sBase":
            features['sBase'] = features.get("sBase",0) + 1
        elif val == "s":
            features['s'] = features.get("s",0) + 1
        elif val == "sMineral":
            features['sMineral'] = features.get("sMineral",0) + 1
        elif 'hotkey' in val:
            features[val] = features.get(val,0) + 1
        else:
            features['apm'] = features.get("apm",0) + 1
            lastFrame = int(val)
    features['apm'] = features['apm'] / (lastFrame/60)
    #print(features)
    test_features.append(features)

100%|██████████| 1024/1024 [00:30<00:00, 49.36it/s]


In [50]:
with open('features.csv', 'w') as outfile:
    fp = csv.DictWriter(outfile, all_features[0].keys())
    fp.writeheader()
    fp.writerows(all_features)

In [6]:
with open('test_features.csv', 'w') as outfile:
    fp = csv.DictWriter(outfile, test_features[0].keys())
    fp.writeheader()
    fp.writerows(test_features)